In [7]:
import pandas as pd
import numpy as np
import re

In [8]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\专利.xlsx',index_col=0)

In [9]:
data.drop_duplicates(keep='first',inplace=True)

In [10]:
data=data.replace(['发明公布更正','实用新型更正','外观设计更正','发明授权更正'],np.nan)
data=data.dropna()
data=pd.get_dummies(data['专利类型'],prefix='专利类型')
data=data.groupby('企业编号').sum().T

In [11]:
data

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
专利类型_发明专利,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
专利类型_发明公布,4.0,1.0,109.0,1.0,2.0,7.0,634.0,155.0,0.0,13.0,...,2.0,42.0,17.0,16.0,27.0,9.0,2.0,7.0,5.0,1.0
专利类型_发明授权,0.0,0.0,59.0,0.0,1.0,4.0,241.0,65.0,0.0,2.0,...,21.0,102.0,25.0,41.0,180.0,27.0,4.0,16.0,22.0,14.0
专利类型_外观设计,0.0,0.0,314.0,4.0,0.0,2.0,1.0,0.0,0.0,1.0,...,13.0,4.0,2.0,15.0,37.0,18.0,0.0,6.0,0.0,18.0
专利类型_实用新型,0.0,1.0,700.0,1.0,3.0,6.0,508.0,127.0,2.0,1.0,...,8.0,36.0,1.0,6.0,48.0,16.0,6.0,0.0,15.0,1.0
